# Multiple Linear Regression

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('HW2_data.csv')
print(df.shape)
df.head()

(30, 4)


,y,x1,x2,x3
0,-1.471961,1,-0.626454,-0.383425
1,4.585642,1,1.595281,1.215549
2,3.113337,1,0.487429,0.562698
3,1.018829,1,-0.305388,0.239762
4,-0.216829,1,-0.621241,-1.099278


Use the provided data set to build a multiple linear regression model

$$\large y = X \beta + \sigma \epsilon $$

1) Find the estimator $\hat\beta = (\hat{\beta_1}, \hat{\beta_2}, \hat{\beta_3})$ and the distribution of  $\hat{\beta_3}$

In [6]:
x1 = df['x1'].to_numpy()
x2 = df['x2'].to_numpy()
x3 = df['x3'].to_numpy()
y = df['y'].to_numpy()

# Get X Matrix
X = np.stack((x1,x2,x3), axis=1)
X[:10]

array([[ 1.        , -0.62645381, -0.38342467],
       [ 1.        ,  1.5952808 ,  1.21554889],
       [ 1.        ,  0.48742905,  0.56269775],
       [ 1.        , -0.30538839,  0.23976248],
       [ 1.        , -0.62124058, -1.09927837],
       [ 1.        , -0.04493361, -0.03631061],
       [ 1.        ,  0.8212212 ,  0.75302523],
       [ 1.        ,  0.7821363 ,  0.56986491],
       [ 1.        ,  0.61982575,  0.4170394 ],
       [ 1.        , -1.47075238, -1.17297169]])

In [7]:
def find_estimators(X,y):
    
    a1 = np.matmul(X.T, X)
    inv_a1 = np.linalg.inv(a1)
    a2 = np.matmul(inv_a1, X.T)
    res = np.matmul(a2, y)
    return res

In [8]:
b1, b2, b3 = find_estimators(X, y)
b1, b2, b3

(0.9925764865075013, 2.0805342305653247, 0.7053498780401117)